#By Knowing this notebook, we can become a eligible \"DATA INGESTION DEVELOPER\"

#### First Let's understand the basic Catalog + Volume Feature of Databricks
Filesystem Hierarchy of Volume in Databricks (DBFS)?<br>
Catalog Menu -> Catalog(Workspace)/schema(database)/volume(Datalake (DBFS))/folder/mere data files<br>
Tables Hierarchy of Databricks?<br>
Catalog Menu-> Catalog(Workspace)/schema(database)/tables(Lakehouse (Delta formatted DBFS))/structure+data(dbfs filesystem/some other filesystems)<br>

#####Create - Catalog(Workspace)/schema(database)/volume(Datalake (DBFS))/folder/mere data files

####If we need to create schema/volume/folder programatically, follow the below steps

In [0]:
%sql
CREATE CATALOG IF NOT EXISTS pmadhan_catalog;
CREATE SCHEMA IF NOT EXISTS pmadhan_catalog.pmadhan_schema;
CREATE VOLUME IF NOT EXISTS pmadhan_catalog.pmadhan_schema.pmadhan_volume;

####Create Directory

In [0]:
dbutils.fs.mkdirs("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1")

####Create some sample data

In [0]:
sampledata="sample data to display"
dbutils.fs.put("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/samplefile.txt",sampledata,True)
dbutils.fs.ls("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1")
print(dbutils.fs.head("dbfs:///Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/samplefile.txt"))

#Spark SQL
###1.E(Extract)
L(Load)<br>
Inbuilt libraries sources/targets & Inbuilt data Formats<br>
2. Bread & Butter (T(Transformation) A(Analytical))<br>

#Learn How to Create Dataframes from filesystem using different options
Download the data from the below drive url <br>
https://drive.google.com/drive/folders/1Tw7V9eBtUxy0xQMW38z3-bzWI_ewzLm6?usp=drive_link

###How Do We Write a Typical Spark Application (Core(Obsolute),SQL(Important),Streaming(Mid level important)) <br>
####Before we Create Dataframe/RDD, what is the prerequisite? We need to create spark session object by instantiating sparksession class (by default databricks did that if you create a notebook)

In [0]:
from pyspark.sql.session import SparkSession
spark = SparkSession.builder.appName("Spark DataFrames").getOrCreate()

Create a DBFS volume namely commondata and upload the above data in that volume <br>
What are other FS uri's (Uniform Resource Identified) available? file://, hdfs://, dbfs://, gs://, s3://, adls://, blob:// <br>
The complete/absolute path with uri prefixed is called url (Uniform Resource Locator)...<br>
This is a FS URL - dbfs:///Volumes/we47catalog/we47schema/we47_volume/we47_dir1/samplefile.txt

#### How to Read/Extract the data from the filesytem and load it into the distributed memory for further processing/load - using diffent methodologies/options from different sources(fs & db) and different builtin function formats (csv/json/orc/parquet/delta/tables)

In [0]:
df1=spark.read.csv("dbfs:///Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/samplefile.txt")
df1.show(20,False)#this show is a DSL function (works only in apache spark), pass number of rows to display and truncate of data upto 20 chars true/false

In [0]:
df1=spark.read.csv("dbfs:///Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custs")
#df1.show(20,False)
df1.printSchema()

####CSV Function complete options
def csv(path: PathOrPaths, schema: Optional[Union[StructType, str]]=None, sep: Optional[str]=None, encoding: Optional[str]=None, quote: Optional[str]=None, escape: Optional[str]=None, comment: Optional[str]=None, header: Optional[Union[bool, str]]=None, inferSchema: Optional[Union[bool, str]]=None, ignoreLeadingWhiteSpace: Optional[Union[bool, str]]=None, ignoreTrailingWhiteSpace: Optional[Union[bool, str]]=None, nullValue: Optional[str]=None, nanValue: Optional[str]=None, positiveInf: Optional[str]=None, negativeInf: Optional[str]=None, dateFormat: Optional[str]=None, timestampFormat: Optional[str]=None, maxColumns: Optional[Union[int, str]]=None, maxCharsPerColumn: Optional[Union[int, str]]=None, maxMalformedLogPerPartition: Optional[Union[int, str]]=None, mode: Optional[str]=None, columnNameOfCorruptRecord: Optional[str]=None, multiLine: Optional[Union[bool, str]]=None, charToEscapeQuoteEscaping: Optional[str]=None, samplingRatio: Optional[Union[float, str]]=None, enforceSchema: Optional[Union[bool, str]]=None, emptyValue: Optional[str]=None, locale: Optional[str]=None, lineSep: Optional[str]=None, pathGlobFilter: Optional[Union[bool, str]]=None, recursiveFileLookup: Optional[Union[bool, str]]=None, modifiedBefore: Optional[Union[bool, str]]=None, modifiedAfter: Optional[Union[bool, str]]=None, unescapedQuoteHandling: Optional[str]=None) -> 'DataFrame'"

####If I don't use any options in this csv function, what is the default functionality?
 #1. By default it will consider ',' as a delimiter (sep='~') <br>
#2. By default it will use _c0,_c1..._cn it will apply as column headers (header=True or toDF(\"\",\"\",\"\") or we have more options to see further) <br>
#3. By default it will treat all columns as string (inferSchema=True or we have more options to see further) <br>
#display with produce output in a dataframe format <br>
#display with produce output in a beautified table format, specific to databricks

**Sample data of custs_1:** <br>
4000001,Kristina,Chung,55,Pilot<br>
4000002,Paige,Chen,77,Teacher <br>
**Sample data of custs_header:**<br>
custid,fname,lname,age,profession<br>
4000001,Kristina,Chung,55,Pilot<br>
4000002,Paige,Chen,77,Teacher

In [0]:
#1. Header Concepts (Either we have define the column names or we have to use the column names from the data)
#Important option is...
#By default it will use _c0,_c1..._cn it will apply as column headers, but we are asking spark to take the first row as header and not as a data?

In [0]:
#We use toDF if we want to add column name explicitly, if the data doesnt has column names or if we want to change the column name the data is provided with\n",
df1=spark.read.csv(path="/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custs").toDF("id","fname","lname","age","profession")
df1.printSchema()
df1.show(2)
display(df1.limit(10))#Display is a special function in databricks, help us display in a table format with a limit that we can define to control the rows"

In [0]:
#We use header option to consider the column name from the data itself\n",
df1=spark.read.csv(path="/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custs_header",header=True)
#We use header option to consider the column name from the data itself and we can override the source given columns to our custom column names also by using header+ toDF()...
print(df1.count())
df1.printSchema()
df1.show(2)
df1=spark.read.csv(path="/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custs_header",header=True).toDF("id","fname","lname","age","occupation")
print(df1.count())
df1.printSchema()
print(df1.count())
df1.printSchema()
df1.show(2)
#display(df1.limit(10))#Display is a special function in databricks, help us display in a table format with a limit that we can define to control the rows"

In [0]:
#2. Printing Schema (equivalent to describe table)
df1.printSchema()

In [0]:
#3. Inferring Schema 
# (Performance Consideration: Use this function causiously because it scans the entire data by immediately evaluating and executing
# hence, not good for large data or not good to use on the predefined schema dataset)
#sample data
#4004979,Tara,Drake,32,
#4004980,Earl,Hahn,34,Human resources assistant
#4004981,Don,Jones,THIRTY SIX,Lawyer

Performance Importance: Though inferSchema has to be used causiously, we can improve performance by using an option to reduce the data scanned for large data..

In [0]:
df1=spark.read.csv(path="/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custs_header",header=True,inferSchema=True)
df1.printSchema()

In [0]:
#4. Using delimiter or seperator option
df1=spark.read.csv(path="/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custs_header_oth_del",header=True,inferSchema=True,sep='~')

#Do we need to put all these efforts?? No
#df1.createOrReplaceTempView("view1")
#spark.sql("select split(`custid~fname~lname~age~profession`,'~') lst_of_cols from view1").createOrReplaceTempView("view2")
#spark.sql("select lst_of_cols[0] id,lst_of_cols[1] fname,lst_of_cols[2] lname,lst_of_cols[3] age,lst_of_cols[4] profession from view2").show(2)
df1.printSchema()
df1.show()

In [0]:

#5. Using different options to create dataframe with csv and other module... (2 methodologies (spark.read.inbuiltfunction or spark.read.format(anyfunction).load("path")) with 3 ways of creating dataframes (pass parameters to the csv()/option/options))
#Methodology #1 (spark.read.inbuiltfunction) we use to create dataframe from builtin sources
df1=spark.read.csv(path="/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custs_header_oth_del",header=True,inferSchema=True,sep='~')
print(df1.count())
#Methodology #2 (spark.read.format('anysources/anyformats').load()) we use to create dataframe from builtin sources
df1_bq=spark.read.format("csv").load("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custs_header")
print(df1_bq.count())

#Mostly we use option or options for external sources, not for builtin sources...
#option can be used for 1 or 2 option...
df1=spark.read.option("header","true").option("sep","~").csv(path="/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custs_header_oth_del")
#or
df1_bq=spark.read.option("header","true").format("csv").load("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custs_header")
print(df1_bq.count())
#options can be used for multiple options in one function as a parameter...
df1=spark.read.options(header=True,sep="~",inferSchema=True).csv(path="/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custs_header_oth_del")
print(df1.count())
#or
df1_bq=spark.read.options(header=True,sep="~",inferSchema=True).format("csv").load("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custs_header")
print(df1_bq.count())

Generic way of read and load data into dataframe using fundamental options from BUILT in sources (csv/orc/parquet/xml/json/table) (inferschema, header, sep)

In [0]:
#Methodology #1 (spark.read.inbuiltfunction) we use to create dataframe from builtin sources
df1=spark.read.csv(path="/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custs_header_oth_del",header=True,inferSchema=True,sep='~')
print(df1.count())

Generic way of read and load data into dataframe using extended options from EXTERNAL sources (bigquery/redshift/athena/synapse) (tmpfolder, access controls)

In [0]:
#options can be used for multiple options in one function as a parameter...
#Methodology #2 (spark.read.format('anysources/anyformats').load()) we use to create dataframe from builtin sources
df1_bq=spark.read.option("header","true").option("inferSchema","true").format("csv").load("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custs_header")
df1_bq=spark.read.options(header=True,sep="~",inferSchema=True).format("csv").load("/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custs_header")
print(df1_bq.count())

Reading data from multiple files & Multiple Path (We still have few more options)

In [0]:
df1_multiple_files=spark.read.csv(path="/Volumes/workspace/default/volume_pmadhan_datalake/custs*",inferSchema=True).toDF("id","fname","lname","age","prof")
print(df1_multiple_files.count())
     

In [0]:
#What if we have files in multiple names in multiple paths... Below code can read data from multiple paths and multiple files...
#Caveate: while reading from multiple csv files.. data patterns like type of column and number of columns should always be same
df1_multiple_files=spark.read.csv(path=["/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1//custs*","/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1//prospect*"],inferSchema=True).toDF("id","fname","lname","age","prof")
print(df1_multiple_files.count())

What if while reading from multiple csv files.. data patterns like type of column and number of columns are not same? We learn this using some feature called schema evolution using few functions like mergeSchema & unionByName in Transformation part..

**Requirement: I am getting data from different source systems of different regions (NY, TX, CA) into different landing pad (locations), how to access this data?**

In [0]:
#recursiveFileLookup is a parameter that can be used to read data from multiple subdirectories
#pathGlobFilter is the parameter used for matching a given pattern under the main/subdirectories
df1_different_subdirs=spark.read.csv("/Volumes/workspace/default/volume_pmadhan_datalake/",recursiveFileLookup=True,pathGlobFilter="custs*",header=True).toDF("id","fname","lname","age","prof")
df1_different_subdirs.count()

### Provide schema with SQL String or programatically (very very important)
We learned so far to define column names explicitly(toDF) or define column names implicitly(header) or infer schema <br>
If we want to handle both column names and structure(schema) explicitly in one shot, we have 2 methods to achieve that... <br>
df1_different_subdirs=spark.read.csv(\"/Volumes/workspace/default/volumewe47_datalake/wd36_source/\",header=True,inferSchema=True)<br>
[PySpark SQL Datatypes](https://spark.apache.org/docs/latest/sql-ref-datatypes.html) <br>
[Data Types](https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/data_types.html)<br>

**###To provide schema (columname & datatype), what are the 2 basic options available that we learned so far ? inferSchema/toDF<br>
  ###We are going to learn additionally 2 more options to handle schema (colname & datatype)?<br>
  ###1. Using simple string format of define schema.<br>
  ###IMPORTANT: 2. Using structure type to define schema.<br>**

In [0]:

###Option 1. Using simple string format of define schema. (least used because it will not support some complex formats)
datastruct="id int,fname string,lname string, age int,prof string"
df0=spark.read.csv(path="/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custs1",sep="~",inferSchema=True,header=False)
df0.display()
df1=spark.read.schema(datastruct).csv(path="/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custs1",sep="~",Header=True)#we didnt use inferSchema or .toDF
df1.printSchema()
df1.show(3)

In [0]:
#IMPORTANT: 2. Using structure type to define schema.\n",
#For the given data \"id,fname,lname,age,prof\" i am going to use StructType(StructField(\"id\",IntegerType()),StructField(\"fname\",StringType())...)\n",
#Pattern: StructType([StructField(\"colname\",DataType()),StructField(\"colname\",DataType())......])\n",
#StructType - 1 time\n",
#StructField - total number of fields time\n",
#IntegerType - 2 times\n",
#StringType - 3 times\n",
#4000001|Kristina|Chung|55|Pilot|\n",
from pyspark.sql.types import StructType,StructField,IntegerType,StringType
custom_schema=StructType([StructField("id",IntegerType(),False),StructField("fname",StringType()),StructField("lname",StringType()),StructField("age",IntegerType()),StructField("profession",StringType())])
df1=spark.read.schema(custom_schema).csv(path="/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/custs1",sep='~',header=True)#we didnt use inferSchema or .toDF\n",
df1.printSchema()
df1.show(3)

###Reading data from different builtin formats (csv, json, xml, orc, parquet, delta, table)

####Reading data from csv

In [0]:
# unable to find files in this location
df1=spark.read.csv("/Volumes/workspace/default/volumewe47_datalake/serialized_compressed_data_sources/csv_targetdata",header=True,sep='~')
 #/Volumes/pmadhan_catalog/pmadhan_schema/pmadhan_volume/directory1/
display(df1.limit(10))

####Reading data from json

In [0]:
df_json=spark.read.json("/Volumes/workspace/default/volumewe47_datalake/serialized_compressed_data_sources/json_targetdata")
display(df_json.take(10))

####Reading data from xml

In [0]:
df_xml=spark.read.xml("/Volumes/workspace/default/volumewe47_datalake/serialized_compressed_data_sources/xml_targetdata",rowTag="customer")
display(df_xml.take(10))

####Reading data from Serialized format - orc

In [0]:
df_orc=spark.read.orc("/Volumes/workspace/default/volumewe47_datalake/serialized_compressed_data_sources/orc_targetdata")
display(df_orc.take(10))

####Reading data from Serialized format - parquet

In [0]:
df_parquet=spark.read.parquet("/Volumes/workspace/default/volumewe47_datalake/serialized_compressed_data_sources/parquet_targetdata") #DSL FBP methodology
display(df_parquet.take(10))

####Reading data from Serialized format - delta(parquet)

In [0]:
df_delta_parquet=spark.read.format("delta").load("/Volumes/workspace/default/volumewe47_datalake/serialized_compressed_data_sources/delta_targetdata")
display(df_delta_parquet.take(10))

####Reading data from table - delta

In [0]:
df_delta_table=spark.read.table("default.cust")
df2=df_delta_table.select("customerid","BranchID","Address","DateOfBirth").filter("branchid=115")#Domain Specific Lang Function based programming
display(df2.take(10))
#or directly write query
df1=spark.sql("select customerid,branchid,address,dateofbirth from default.cust where branchid=115") #SQL Declarative Query
display(df1.take(10))

 What are all the overall functions & options we used/learned in this notebook, for learning fundamental spark csv read operations? <br>
    - **The concept of catalog & volume...** <br>
    - spark.read.csv(path=\"\") <br>
    - spark.read.csv(path=[]) <br>
    - **Basic (important) options** - inferSchema=True, sep='~', header=True, toDF(), schema <br>
    - **Additional options** - recursiveFileLookup, pathGlobalFilter.<br>
    - **Other options based on questions** - linesep, sampling ratio, unionByName,allowMissingColumns.<br>
    - spark.read.format(\"\").load()<br>
    - spark.read.option(k,v).format(\"\").load()<br>
    - spark.read.options(k=v,k=v).format(\"\").load()<br>
    - Custom schema:<br>
    - strstruct=\"colname datatype,colname datatype <br>
    - strcustomstruct=StructType([StructField(\"colname\",DataType()),....]) <br>
    - How to read data from different builtin formats (csv, json, xml, orc, parquet, delta, table) <br>